# Making a cubic camembert

The moonshot: I have designed a camembert contact map. Let's see if it actually works!

In [109]:
import numpy as np
from json_dump import *
import contact_utils as cu
import config as cfg
from pathlib import Path
from geometry.cubic import CubicGeometry
from plotting.plot_cubic import plot_cubes_from_simulation_results

In [110]:
### Key parameter: how we will name this run
run_name = "01_camembert_test"

### Define model parameters

model_params = {}

# ---------- LATTICE OPTIONS ----------

# Options:
# "chain", "square", "triangular", "cubic", "bcc", "fcc"
model_params["lattice_name"] = "cubic"

# Lattice dimensions
model_params["lx"] = 18
model_params["ly"] = 18  # Has to be 1 for chain
model_params["lz"] = 18  # Has to be 1 for square & triangular

In [111]:
# ---------- MODEL PARAMETERS ----------

# Number of particle types
model_params["n_types"] = 1

# Number of particles of each type
model_params["n_particles"] = [200]

# Couplings is its own beast. Should be gotten with the appropriate helper
# function.

## Important note: for 3D particles, do not use the get_matrix and set_contact utils!

A lot of face pairs are equivalent under rotation for cubes (e.g. (0, 0), (1, 3), (2, 2), (3, 1)).
When using a full contact matrix and feeding it to the wrapper using `set_single_species_contacts`, if all the equivalent pairs don't
have the same matrix coefficient, some of them will be overwritten by other values, and the contacts will be wrong.

Better to do things the way they are done below

In [135]:
crystal_energy = -18.7
# crystal_energy = 0
# defect_energy = -10
defect_energy = crystal_energy / 0.5
repel_energy = 10000
# crystal_energy = repel_energy

In [136]:
get_opposite_face = CubicGeometry().get_opposite_face
# First, create a cmap_wrapper object with the appropriate number of particle types
cmap_wrapper = cu.ContactMapWrapper.cubic(
    n_types = model_params["n_types"], init_energy = repel_energy)

# Setting crystalline interactions
for i in range(3):
    face = 4 * i
    opposite_face = get_opposite_face(face)
    cmap_wrapper[face, opposite_face] = crystal_energy

In [160]:
# Setting camembert interactions

# First, let's tabulate the contacts along each circulation line
# of internal axes
contacts_Rplus = [
    (4, 13),
    (8, 13),
    (12, 17),
    (12, 23),
]

contacts_Gplus = [
    (0, 17),
    (8, 16),
    (12, 17),
    (16, 20)
]

contacts_Bplus = [
    (0, 21),
    (4, 22),
    (12, 23),
    (16,20)
]

contacts_Rminus = [
    (0, 9),
    (0, 17),
    (0, 21),
    (0, 7)
]

contacts_Gminus = [
    (4, 22),
    (4, 13),
    (4, 10),
    (0, 7),
]

contacts_Bminus = [
    (0, 9),
    (4, 10),
    (8, 13),
    (8, 16)
]

In [161]:
Rplus_flag = True
Gplus_flag = True
Bplus_flag = True

Rminus_flag = True
Gminus_flag = True
Bminus_flag = True

camembert_contacts = []
if Rplus_flag:
    camembert_contacts.extend(contacts_Rplus)
if Gplus_flag:
    camembert_contacts.extend(contacts_Gplus)
if Bplus_flag:
    camembert_contacts.extend(contacts_Bplus)
if Rminus_flag:
    camembert_contacts.extend(contacts_Rminus)
if Gminus_flag:
    camembert_contacts.extend(contacts_Gminus)
if Bminus_flag:
    camembert_contacts.extend(contacts_Bminus)


for contact in camembert_contacts:
    cmap_wrapper[contact] = defect_energy

# And process the couplings into flattened form
model_params["couplings"] = cmap_wrapper.get_formatted_couplings()

In [154]:
# Initialization option
model_params["initialize_option"] = "random"

# If "initialize_option" is set to "from_file", we must specify the location of
# the input file
# model_params["state_input"] = str(cfg.structures_path/"final_structure.dat")

# Options for average collection

model_params["state_av_option"] = True
model_params["e_av_option"] = True

if model_params["state_av_option"]:
    state_path = Path("./data/"+run_name+"/average_state")
    state_path.mkdir(parents = True, exist_ok = True)
    model_params["state_av_output"] =  str(state_path.resolve()) + "/"

if model_params["e_av_option"]:
    energy_path = Path("./data/"+run_name+"/average_energy")
    energy_path.mkdir(parents = True, exist_ok = True)
    model_params["e_av_output"] = str(energy_path.resolve())  + "/"

In [155]:
# Pick the probabilities of different moves. Has to sum to 1.
# Options:
""""
    "swap_empty_full",
    "swap_full_full",
    "rotate",
    "mutate",
    "rotate_and_swap_w_empty"
"""

moves_dict = {}
moves_dict["swap_empty_full"] = 1/3
moves_dict["rotate"] = 1/3
moves_dict["rotate_and_swap_w_empty"] = 1/3

model_params["move_probas"] = moves_dict

make_json_file(model_params, cfg.input_path/"model_params.json")

In [156]:
### Define mc parameters

mc_params = {}

# Number of MC steps used for equilibration
mc_params["mcs_eq"] = 1000

# Number of MC steps used for averaging
mc_params["mcs_av"] = 10

# Type of cooling schedule
# if exponential chosen: specify log10(T) as initial and final temperatures
mc_params["cooling_schedule"] = "exponential"

# Initial annealing temperature
mc_params["Ti"] = 2

# Final annealing temperature
mc_params["Tf"] = 0

# Number of annealing steps
mc_params["Nt"] = 40

# Option to collect state checkpoints at the end of each temperature cycle
mc_params["checkpoint_option"] = True

# If checkpoint is True, we need to provide the output address for the 
# checkpoint files
structures_path = Path("./data/"+run_name+"/structures")
structures_path.mkdir(parents = True, exist_ok = True)

if mc_params["checkpoint_option"]:
    mc_params["checkpoint_address"] = str(structures_path.resolve())+"/"

# Output location of the final state configuration (must end with "/")
mc_params["final_structure_address"] = str(structures_path.resolve())+"/"

make_json_file(mc_params, cfg.input_path/"mc_params.json")

In [157]:
# Run the simulation from inside the notebook.
# Note that the program only writes to the cell output after it is done running.
# If you want to follow the excecution in real time, run the program from the command line!

# Parameter in run_sim is a boolean flag, which will overwrite the simulation results if set to True

cfg.run_simulation(True)

At least one of the output folders is already populated!
overwrite flag set to True: running anyway.
Program runs!
5832
Printing current system state
Number of particle types: 1
Number of particles of each type: {200, }
Printing interactions structure
Printing coupling matrix: {10000, 10000, 10000, 10000, 10000, 10000, 10000, -37.4, 10000, -37.4, 10000, 10000, -18.7, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, -37.4, 10000, -37.4, 10000, 10000, 10000, 10000, -18.7, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, -18.7, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, -18.7, -37.4, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 10000, 10000, 10000, -37.4, 10000, 10000, 10000, 

In [159]:
# Plotting module creates a blender file which you have to open yourself

blend_file_path = f"3dFigures/02_camembert_contacts"
if Rplus_flag:
    blend_file_path += "_Rplus"
if Gplus_flag:
    blend_file_path += "_Gplus"
if Bplus_flag:
    blend_file_path += "_Bplus"
if Rminus_flag:
    blend_file_path += "_Rminus"
if Gminus_flag:
    blend_file_path += "_Gminus"
if Bminus_flag:
    blend_file_path += "_Bminus"
blend_file_path += ( f"_crystal_{crystal_energy:.02f}"
f"_rep_{repel_energy:.02f}_defect_{defect_energy:.02f}.blend")

plot_cubes_from_simulation_results(struct_folder = str(structures_path.resolve()), fig_file = blend_file_path)

200


Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
/var/folders/v3/gbqhvbxs6nz1258v43493tx40000gp/T/ipykernel_70733/2060227659.py:21: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  plot_cubes_from_simulation_results(struct_folder = str(structures_path.resolve()), fig_file = blend_file_

umbered.obj' took 0.80 ms
OBJ import of 'one_cube_numbered.obj' took 0.72 ms
OBJ import of 'one_cube_numbered.obj' took 1.08 ms
OBJ import of 'one_cube_numbered.obj' took 1.03 ms
OBJ import of 'one_cube_numbered.obj' took 0.81 ms
OBJ import of 'one_cube_numbered.obj' took 0.59 ms
OBJ import of 'one_cube_numbered.obj' took 0.69 ms
OBJ import of 'one_cube_numbered.obj' took 0.77 ms
OBJ import of 'one_cube_numbered.obj' took 0.61 ms
OBJ import of 'one_cube_numbered.obj' took 0.93 ms
OBJ import of 'one_cube_numbered.obj' took 0.58 ms
OBJ import of 'one_cube_numbered.obj' took 0.74 ms
OBJ import of 'one_cube_numbered.obj' took 0.68 ms
OBJ import of 'one_cube_numbered.obj' took 0.67 ms
OBJ import of 'one_cube_numbered.obj' took 0.68 ms
OBJ import of 'one_cube_numbered.obj' took 0.73 ms
OBJ import of 'one_cube_numbered.obj' took 0.68 ms
OBJ import of 'one_cube_numbered.obj' took 1.15 ms
OBJ import of 'one_cube_numbered.obj' took 0.75 ms
OBJ import of 'one_cube_numbered.obj' took 0.89 ms
OBJ i

Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/

Info: Saved "02_camembert_contacts_Rplus_Gplus_Bplus_Rminus_Gminus_Bminus_crystal_-18.70_rep_10000.00_defect_-37.40.blend"


Once you have ran the above code, you should open the resulting .blend file in blender and see the results for yourself!